In [1]:
import os
import cv2
import numpy as np
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

In [2]:
def load_data(data_dir):
    X = []
    y = []
    for label in os.listdir(data_dir):
        label_dir = os.path.join(data_dir, label)
        for filename in os.listdir(label_dir):
            file_path = os.path.join(label_dir, filename)
            if filename.endswith('.mp4'):
                cap = cv2.VideoCapture(file_path)
                ret, frame = cap.read()
                if not ret:
                    print(f"Não foi possível abrir o vídeo {file_path}")
                    continue
                img = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            else:
                img = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)
                if img is None:
                    print(f"Não foi possível abrir a imagem {file_path}")
                    continue
            img = cv2.resize(img, (64, 64))  # Redimensionar para o tamanho desejado
            X.append(img)
            y.append(label)  # Rotular com o nome da pasta (palavra em Libras)
    return np.array(X), np.array(y)



In [3]:
num_classes = 5

# Definir o modelo CNN
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 1)))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))  # num_classes é o número de palavras em Libras

# Compilar o modelo
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])



In [22]:
# Carregar os dados de treinamento
X_train, y_train = load_data('./pasta2')

# Pré-processamento dos dados
X_train = X_train.reshape(-1, 64, 64, 1)
X_train = X_train.astype('float32') / 255.0

encoder = LabelEncoder()
y_train_encoded = encoder.fit_transform(y_train)

# Converter rótulos inteiros para formato one-hot
y_train_one_hot = to_categorical(y_train_encoded)

# Agora você pode treinar o modelo com y_train_one_hot
model.fit(X_train, y_train_one_hot, epochs=10, batch_size=32)

# Salvar o modelo treinado
model.save('libras_model.h5')
np.save('classes.npy', encoder.classes_)

Epoch 1/10
1/1 [==============================] - 0s 105ms/step - loss: 0.0482 - accuracy: 1.0000
Epoch 2/10
1/1 [==============================] - 0s 7ms/step - loss: 0.0362 - accuracy: 1.0000
Epoch 3/10
1/1 [==============================] - 0s 4ms/step - loss: 0.0273 - accuracy: 1.0000
Epoch 4/10
1/1 [==============================] - 0s 3ms/step - loss: 0.0210 - accuracy: 1.0000
Epoch 5/10
1/1 [==============================] - 0s 4ms/step - loss: 0.0166 - accuracy: 1.0000
Epoch 6/10
1/1 [==============================] - 0s 4ms/step - loss: 0.0130 - accuracy: 1.0000
Epoch 7/10
1/1 [==============================] - 0s 5ms/step - loss: 0.0102 - accuracy: 1.0000
Epoch 8/10
1/1 [==============================] - 0s 5ms/step - loss: 0.0081 - accuracy: 1.0000
Epoch 9/10
1/1 [==============================] - 0s 4ms/step - loss: 0.0065 - accuracy: 1.0000
Epoch 10/10
1/1 [==============================] - 0s 4ms/step - loss: 0.0054 - accuracy: 1.0000
